# Generate synthetic data for the ml4h featured workspace

This notebook is a companion to the ml4h Terra featured workspace, but it is not included in that workspace because it contains the *answers* to the *quiz*. It is checked into GitHub so that people can see how this static data was generated, and also offer improvements!

# Setup 

In [ ]:
lapply(c('hrbrthemes', 'sn'),
       function(pkg) { if(! pkg %in% installed.packages()) { install.packages(pkg)} } )

In [ ]:
library(bigrquery)
library(hrbrthemes)
library(sn)
library(tidyverse)

In [ ]:
BILLING_PROJECT_ID <- Sys.getenv('GOOGLE_PROJECT')

theme_set(theme_ipsum(base_size = 16) + theme(axis.title.x = element_text(size = 16),
                                              axis.text.x = element_text(size = 14),
                                              axis.title.y = element_text(size = 16),
                                              axis.text.y = element_text(size = 14)))
options(repr.plot.width = 14, repr.plot.height = 10)

# Generate synthetic data 

In [ ]:
set.seed(2)
NUM_SAMPLES_TO_GENERATE <- 20000
MIN_AGE <- 40
MAX_AGE <- 80

In [ ]:
generated_data <- tibble(
    sample_num = seq(1, NUM_SAMPLES_TO_GENERATE),
    # Randomly assign fake samples to a percentile. We'll anchor on this for other measures.
    percentile = as.integer(runif(n = NUM_SAMPLES_TO_GENERATE, min = 0, max = 100)),
    # Assign roughly half of the samples to each sex_at_birth.
    sex_at_birth = sample(x = c('male', 'female'),
                          size = NUM_SAMPLES_TO_GENERATE,
                          replace = TRUE,
                          prob = c(0.47, 0.53)),
    # Uniformly distribute samples across the age range.
    age = as.integer(runif(n = NUM_SAMPLES_TO_GENERATE,
                           min = MIN_AGE,
                           max = MAX_AGE + 1)),
    # Normally distribute BMI http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=21001
    bmi_norm_female = rnorm(n = NUM_SAMPLES_TO_GENERATE, mean = 27.03082, sd = 5.207417),
    bmi_norm_male = rnorm(n = NUM_SAMPLES_TO_GENERATE, mean = 27.82755, sd = 4.263545),
    # Use proportions from http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=6024
    ecg_exercise_program = sample(
        x = c('Minimal risk, cycle at 50% of max work load',
              'Small risk, cycle at 35% of max work load',
              'Medium risk, cycle at constant level',
              'High risk, take measurement at rest only',
              'ECG not to be performed'),
        size = NUM_SAMPLES_TO_GENERATE,
        replace = TRUE,
        prob = c(73554, 11439, 2874, 10173, 1481) / 96550 ),
    # Exponentially and uniformly distribute Proton density visceral adipose fraction
    # http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22402
    pdff_exp = rexp(n=NUM_SAMPLES_TO_GENERATE, rate = 1),
    pdff_unif = runif(n = NUM_SAMPLES_TO_GENERATE, min = 0, max = 20),
    # Normally distribute QT interval http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22331
    qt_interval_actual = rnorm(n = NUM_SAMPLES_TO_GENERATE,
                               mean = 418.036,
                               sd = 32.4123),
    # Skew-normal distribute P axis http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22335
    p_axis_actual = rsn(n = NUM_SAMPLES_TO_GENERATE, dp = cp2dp(c(49.6788, 23.3791, -0.8), 'SN')),
) %>%
mutate (
    sample_id = str_glue('fake_{sample_num}'),
    # Mix the two distributions based on gender.
    bmi_norm = ifelse(sex_at_birth == 'female', bmi_norm_female, bmi_norm_male),
    # Mix the two distributions to add a long tail to pdff.
    pdff_actual = ifelse(percentile <= 18, pdff_unif, pdff_exp)
) %>%
rowwise() %>% # We need 'rowwise' to get the single age per person for the probability in the bmi_skew.
mutate(
    # Skew BMI a bit lower for younger samples and higher for older samples.
    bmi_skew = runif(n = 1, min = 1.0, max = 3.0) * sample(x = c(1, -1), size = 1, prob = c(age/MAX_AGE, 1 - age/MAX_AGE)),
    bmi = bmi_norm + bmi_skew,
    # Shift extreme values back towards the mean.
    bmi = ifelse(bmi < 14, bmi + 5, bmi),
    # Add some jitter to the prediction, more jitter for lower effort bike ECGs.
    qt_interval_prediction = qt_interval_actual + case_when(
        ecg_exercise_program == 'Minimal risk, cycle at 50% of max work load' ~ rnorm(n = 1, mean = 0, sd = 1.0),
        ecg_exercise_program == 'Small risk, cycle at 35% of max work load' ~ rnorm(n = 1, mean = 0, sd = 3.0),
        TRUE ~ rnorm(n = 1, mean = 0, sd = 10.0)),
    # Add some jitter to the prediction, more jitter for larger BMI.
    p_axis_prediction = p_axis_actual + case_when(
        percentile <= 10 ~ rnorm(n = 1, mean = 0, sd = 3.0),
        percentile >= 90 ~ rnorm(n = 1, mean = 0, sd = 1.0),
        bmi > 30 ~ rnorm(n = 1, mean = 0, sd = 3.0),
        TRUE ~ rnorm(n = 1, mean = 0, sd = 1.0)),
    # Add some jitter to the prediction, more jitter for both people over weight,
    # very little jitter for those underweight.
    pdff_prediction = pdff_actual + case_when(
        bmi > 30 ~ rsn(n = 1, dp = cp2dp(c(2.5, 2.0, 0.8), 'SN')),
        bmi < 18.5 ~ rsn(n = 1, dp = cp2dp(c(0, 0.4, -0.8), 'SN')),
        TRUE ~ rsn(n = 1, dp = cp2dp(c(0.0, 1.0, 0.0), 'SN')),
    ),
    # Add all the deltas for comparison to BMI and ecg_exercise.
    qt_interval_delta = qt_interval_actual - qt_interval_prediction,
    p_axis_delta = p_axis_actual - p_axis_prediction,
    pdff_delta = pdff_actual - pdff_prediction,
)

In [ ]:
head(generated_data)

# Compare to reported distributions 

## sex at birth

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=31
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c31.png)

In [ ]:
qplot(as_factor(generated_data$sex_at_birth)) + coord_flip()

In [ ]:
ggplot(generated_data, aes(x = sex_at_birth, y = bmi)) + geom_boxplot()

## BMI

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=21001
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c21001.png)

In [ ]:
qplot(generated_data$bmi)

## age

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=21003
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c21003.png)

In [ ]:
qplot(generated_data$age, bins = max(generated_data$age) - min(generated_data$age) + 1)

### Age vs. BMI

BMI increases for a particular person over time, but from the real data, we see that across the population there does not appear to be a correlation between BMI and age.

For now, I left in the skew to a higher BMI with age, but it can be taken out.

In [ ]:
qplot(data = generated_data, x = age, y = bmi_norm) + geom_smooth()

In [ ]:
qplot(data = generated_data, x = age, y = bmi_skew) + geom_smooth()

In [ ]:
qplot(data = generated_data, x = age, y = bmi) + geom_smooth()

In [ ]:
cor(generated_data$age,
    generated_data$bmi,
    use = 'pairwise.complete.obs',
    method = 'spearman')

## Bike speed

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=5985
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c5985.png)

## Program category for ECG during exercise

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=6024
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c6024.png)

In [ ]:
qplot(fct_relevel(generated_data$ecg_exercise_program,
                  'ECG not to be performed',
                  'High risk, take measurement at rest only',
                  'Medium risk, cycle at constant level',
                  'Small risk, cycle at 35% of max work load',
                  'Minimal risk, cycle at 50% of max work load')
     ) + coord_flip()

## P axis 

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22335
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c22335.png)

In [ ]:
qplot(generated_data$p_axis_actual)

In [ ]:
qplot(generated_data$p_axis_prediction)

In [ ]:
cor(generated_data$p_axis_actual, generated_data$p_axis_prediction)

In [ ]:
qplot(generated_data$p_axis_actual, generated_data$p_axis_prediction) +
    geom_abline(intercept = 0, color = 'red')

### P axis delta vs. BMI

In [ ]:
qplot(generated_data$bmi, generated_data$p_axis_delta) + geom_hline(yintercept = 0, color = 'red')

## QT interval 

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22331
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c22331.png)

In [ ]:
qplot(generated_data$qt_interval_actual)

In [ ]:
qplot(generated_data$qt_interval_prediction)

In [ ]:
cor(generated_data$qt_interval_actual, generated_data$qt_interval_prediction)

In [ ]:
qplot(generated_data$qt_interval_actual, generated_data$qt_interval_prediction) +
    geom_abline(intercept = 0, color = 'red')

### QT interval delta vs. BMI 

In [ ]:
qplot(generated_data$bmi, generated_data$qt_interval_delta) + geom_hline(yintercept = 0, color = 'red')

### QT interval delta vs. ECG program category

In [ ]:
generated_data %>%
  mutate(
    ecg_exercise_program = parse_factor(ecg_exercise_program,
                                        levels = c('ECG not to be performed',
                                                   'High risk, take measurement at rest only',
                                                   'Medium risk, cycle at constant level',
                                                   'Small risk, cycle at 35% of max work load',
                                                   'Minimal risk, cycle at 50% of max work load'))
) %>%
  ggplot(aes(x = qt_interval_actual, y = qt_interval_prediction, color = ecg_exercise_program)) +
    geom_point(alpha = 0.5) +
    geom_abline(intercept = 0, color = 'black')

In [ ]:
options(repr.plot.height = 14)

generated_data %>%
  mutate(
    ecg_exercise_program = parse_factor(ecg_exercise_program,
                                        levels = c('ECG not to be performed',
                                                   'High risk, take measurement at rest only',
                                                   'Medium risk, cycle at constant level',
                                                   'Small risk, cycle at 35% of max work load',
                                                   'Minimal risk, cycle at 50% of max work load'))
) %>%
ggplot(aes(x = qt_interval_actual, y = qt_interval_prediction, color = ecg_exercise_program)) +
    geom_point() +
    geom_abline(intercept = 0, color = 'black') +
    facet_grid(rows = vars(ecg_exercise_program))

## Proton density visceral adipose fraction

http://biobank.ndph.ox.ac.uk/showcase/field.cgi?id=22402
![image](http://biobank.ndph.ox.ac.uk/showcase/showcase/graphs/c22402.png)

In [ ]:
options(repr.plot.height = 10)

qplot(generated_data$pdff_actual)

In [ ]:
qplot(generated_data$pdff_prediction)

In [ ]:
cor(generated_data$pdff_actual, generated_data$pdff_prediction)

In [ ]:
qplot(generated_data$pdff_actual, generated_data$pdff_prediction) +
    geom_abline(intercept = 0, color = 'red')

### pdff delta vs. BMI 

In [ ]:
qplot(generated_data$bmi, generated_data$pdff_delta) + geom_hline(yintercept = 0, color = 'red')

# Compare other variables that we might expect to be correlated

TODO: Are there any other variables we might expect to be correlated? e.g., Should any of the actual prediction values be more strongly correlated with BMI?

# Create CSV export

In [ ]:
colnames(generated_data)

In [ ]:
subset_for_export <- generated_data %>%
    # Grab a subset of the columns, renaming a few.
    select(sample_id,
           age,
           bmi,
           sex_at_birth,
           ecg_exercise_program,
           qt_interval_actual,
           qt_interval_prediction,
           p_axis_actual,
           p_axis_prediction,
           proton_density_fat_actual = pdff_actual,
           proton_density_fat_prediction = pdff_prediction
          )

# Take a look at the samples we want to modify.
subset_for_export %>% filter(sample_id %in% c('fake_1', 'fake_2'))

## Inject our manual values.

These are intended to convey the story being told within the other notebooks in the Terra workspace.

In [ ]:
final_export <- subset_for_export %>%
    mutate(
        age = case_when(
            sample_id == 'fake_1' ~ as.integer(70),
            sample_id == 'fake_2' ~ as.integer(74),
            TRUE ~ age  # Keep the same value for everyone else.
        ),
        bmi = case_when(
            sample_id == 'fake_1' ~ 38.1,
            sample_id == 'fake_2' ~ 24.7,
            TRUE ~ bmi  # Keep the same value for everyone else.
        ),
        sex_at_birth = case_when(
            sample_id == 'fake_1' ~ 'male',
            sample_id == 'fake_2' ~ 'male',
            TRUE ~ sex_at_birth  # Keep the same value for everyone else.
        ),
        ecg_exercise_program = case_when(
            sample_id == 'fake_1' ~ 'Small risk, cycle at 35% of max work load',
            sample_id == 'fake_2' ~ 'High risk, take measurement at rest only',
            TRUE ~ ecg_exercise_program  # Keep the same value for everyone else.
        ),
        qt_interval_actual = case_when(
            sample_id == 'fake_1' ~ 374.1,
            sample_id == 'fake_2' ~ 428.7,
            TRUE ~ qt_interval_actual  # Keep the same value for everyone else.
        ),
        qt_interval_prediction = case_when(
            sample_id == 'fake_1' ~ 373.8,
            sample_id == 'fake_2' ~ 304.2,
            TRUE ~ qt_interval_prediction  # Keep the same value for everyone else.
        ),
        p_axis_actual = case_when(
            sample_id == 'fake_1' ~ 8.1,
            sample_id == 'fake_2' ~ 87.0,
            TRUE ~ p_axis_actual  # Keep the same value for everyone else.
        ),
        p_axis_prediction = case_when(
            sample_id == 'fake_1' ~ 10.8,
            sample_id == 'fake_2' ~ 87.3,
            TRUE ~ p_axis_prediction  # Keep the same value for everyone else.
        ),
        proton_density_fat_actual = case_when(
            sample_id == 'fake_1' ~ 15.4,
            sample_id == 'fake_2' ~ 0.69,
            TRUE ~ proton_density_fat_actual  # Keep the same value for everyone else.
        ),
        proton_density_fat_prediction = case_when(
            sample_id == 'fake_1' ~ 12.0,
            sample_id == 'fake_2' ~ 0.72,
            TRUE ~ proton_density_fat_prediction  # Keep the same value for everyone else.
        ),
        # Recompute all the deltas.
        qt_interval_delta = qt_interval_actual - qt_interval_prediction,
        p_axis_delta = p_axis_actual - p_axis_prediction,
        proton_density_fat_delta = proton_density_fat_actual - proton_density_fat_prediction,
    )

In [ ]:
final_export %>% filter(sample_id %in% c('fake_1', 'fake_2'))

In [ ]:
final_export_file = 'synthetic_pheno_and_results.csv'
write_csv(final_export, path = final_export_file)